    1.  Считайте файл и создайте Spark DataFrame.

In [1]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder.appName("CSV to DataFrame").getOrCreate()


csv_file_path = "130/4/4.csv"

df = spark.read.csv(csv_file_path, header=True, inferSchema=True)


df.show()

df.printSchema()

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---+------+
|         feature_1|         feature_2|         feature_3|         feature_4|         feature_5|         feature_6|         feature_7|         feature_8|         feature_9|        feature_10|        feature_11|        feature_12|        feature_13| id|target|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---+------+
|12.131432583642061| 14.78459292345522| 34.45008992992243|11.383921357590463|23.180637702949333| 38.45948351494353| 26.48045333974461|36.824231750331656|29.861555327274413|20.797340757288968|  21.6065766729565|26.7519635

    2.	Подготовьте исходный набор данных для расчета критерия согласия Пирсона (Pearson’s Chi-squared test) 

In [11]:
from pyspark.sql.functions import col, mean

# Заполнение пропущенных значений средним по колонке
columns = df.columns

for column in columns:
    if df.schema[column].dataType.typeName == 'double':
        mean_value = df.select(mean(col(column))).collect()[0][0]
        df = df.fillna(column = mean_value)

columns_to_drop = ["id","target"]

features = df.drop(*columns_to_drop)

features_name = features.columns

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|         feature_1|         feature_2|         feature_3|         feature_4|         feature_5|         feature_6|         feature_7|         feature_8|         feature_9|        feature_10|        feature_11|        feature_12|        feature_13|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|12.131432583642061| 14.78459292345522| 34.45008992992243|11.383921357590463|23.180637702949333| 38.45948351494353| 26.48045333974461|36.824231750331656|29.861555327274413|20.797340757288968|  21.6065766729565|26.751963594682014|13.869156772741839|
| 17

3.	Выполните расчет критерия согласия Пирсона (Pearson’s Chi-squared test)


In [17]:

from pyspark.ml.stat import ChiSquareTest
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import DenseVector

assembler = VectorAssembler(
    inputCols=features_name,
    outputCol="features"
)

df_with_vectors = assembler.transform(df)

chiSqResult = ChiSquareTest.test(df_with_vectors, "features", "target")

chiSqResult.show()

+--------------------+--------------------+--------------------+
|             pValues|    degreesOfFreedom|          statistics|
+--------------------+--------------------+--------------------+
|[0.48669069197573...|[1247, 1247, 1247...|[1248.00000000002...|
+--------------------+--------------------+--------------------+



    4.	Запишите полученные данные в файл (п. 3).

In [21]:
file_path = "130/4/chiSqResult.json"

ps_chiSqResult = chiSqResult.toPandas()

ps_chiSqResult.to_json(file_path)